# CARS recommender system
Implementation of the deep NN model described in the paper "Context-Aware Recommendations Based on Deep
Learning Frameworks".
https://dl.acm.org/doi/10.1145/3386243

Datasets:
- frappe


## Import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # to split dataset
from sklearn.metrics import * # evaluation metrics
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Flatten
from keras.layers import Concatenate
from keras.optimizers import Adam
import matplotlib.pyplot as plt # for creating chart
import requests # for downloading the dataset

## Load and display dataset

In [ ]:
# download the dataset
url = 'https://raw.githubusercontent.com/CriptHunter/tesi/master/CARS_DL/frappe/frappe.csv'
req = requests.get(url, allow_redirects=True)
open('frappe.csv', 'wb').write(req.content)

In [ ]:
# open the dataset
df = pd.read_csv('/content/frappe.csv', sep="\t")
df

In [ ]:
# count unique values for each column
display("------ unique values ------")
display(df.nunique())

# count number of unknown values for each column
display("------ unknown values ------")
display(df.isin(['unknown']).sum(axis=0))

# count number of zero values for each column (for city 0 == unknown)
display("------ zero values ------")
display(df.isin([0]).sum(axis=0))

## Dataset preprocessing

In [ ]:
# log transformation on the raw frequency numbers that represent the applications usage
df['cnt'] = df['cnt'].apply(np.log10)
f"frequency range is {df['cnt'][df['cnt'] == df['cnt'].min()].values[0]} to {df['cnt'][df['cnt'] == df['cnt'].max()].values[0]}"

# delete columns that are not needed
del df['homework']
del df['cost']
del df['city']
del df['isweekend']
del df['country']

In [ ]:
# delete rows where weather is unknown
df = df[df.weather != 'unknown']
df = df.reset_index(drop=True)

# make user and items id start from 0
df.user = pd.factorize(df.user)[0]
df.item = pd.factorize(df.item)[0]

df

In [ ]:
context_labels = ['daytime', 'weekday', 'weather']

# convert categorical data to one-hot encoding
for col in context_labels:
  df = pd.get_dummies(df, columns=[col], prefix = [col])

# new context labels after one-hot encoding are columns from 3 to the end
context_labels = df.columns[3:]
df

In [ ]:
# train and test datasets
train_x, test_x = train_test_split(df, test_size=0.2)

# train and test context features
train_context = pd.concat([train_x.pop(x) for x in context_labels], axis=1)
test_context = pd.concat([test_x.pop(x) for x in context_labels], axis=1)

# train and test values to predict
train_y = train_x.pop('cnt')
test_y = test_x.pop('cnt')

f"train_x: {train_x.shape}   train_y: {train_y.shape}   train_context: {train_context.shape}    test_x: {test_x.shape}   test_y: {test_y.shape}     test_context:   {test_context.shape}"    

In [ ]:
# count number of unique users and items
n_users, n_items = len(df.user.unique()), len(df.item.unique())
n_context = len(context_labels)

# embedding vectors length
n_latent_factors_user = 8
n_latent_factors_item = 12

f'Number of users: {n_users}      Number of apps: {n_items}     Number of context features: {n_context}'

## ECAM NCF

In [ ]:
def ecam_ncf():
    # inputs
    item_input = Input(shape=[1],name='item')
    user_input = Input(shape=[1],name='user')
    context_input = Input(shape=(n_context, ), name='context')

    # Item embedding
    item_embedding_mlp = Embedding(n_items + 1, n_latent_factors_item, name='item_embedding')(item_input)
    item_vec_mlp = Flatten(name='flatten_item')(item_embedding_mlp)
    item_vec_mlp = Dropout(0.2)(item_vec_mlp)

    # User embedding
    user_embedding_mlp = Embedding(n_users + 1, n_latent_factors_user,name='user_embedding')(user_input)
    user_vec_mlp = Flatten(name='flatten_user')(user_embedding_mlp)
    user_vec_mlp = Dropout(0.2)(user_vec_mlp)

    # Concat user embedding,item embeddings and context vector
    concat = Concatenate(name='user_item')([item_vec_mlp, user_vec_mlp, context_input])

    # dense layers
    dense = Dense(8, name='fully_connected_1')(concat)
    batch_1 = BatchNormalization()(dense)
    dense_2 = Dense(4, name='fully_connected_2')(batch_1)
    batch_2 = BatchNormalization()(dense_2)
    dense_3 = Dense(2, name='fully_connected_3')(batch_2)

    # Output
    pred_mlp = Dense(1, activation='relu', name='Activation')(dense_3)

    # make and build the model
    return keras.Model([user_input, item_input, context_input], pred_mlp)

In [ ]:
ecam_ncf = ecam_ncf()
opt = keras.optimizers.Adam(lr = 0.005)
ecam_ncf.compile(optimizer = opt,loss= 'mean_absolute_error', metrics=['mae', 'mse'])

ecam_ncf.summary()
tf.keras.utils.plot_model(ecam_ncf)

In [ ]:
history = ecam_ncf.fit([train_x.user, train_x.item, train_context], train_y, epochs=15, batch_size=128, verbose=1)

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.ylim([0, 1])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)
plot_loss(history)

In [ ]:
# prediction on the test set
pred_y = ecam_ncf.predict([test_x.user, test_x.item, test_context]).flatten()

# chart that show predictions and true values
a = plt.axes(aspect='equal')
plt.scatter(test_y, pred_y)
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [0, 5]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
# evaluation metrics on the test set
rmse = mean_squared_error(test_y, pred_y, squared = False)
mse = mean_squared_error(test_y, pred_y, squared = True)
mae = mean_absolute_error(test_y, pred_y)
f'RMSE = {rmse}    MAE = {mae}    MSE = {mse}'

## ECAM NeuMF

In [ ]:
# latent factors for matrix factorization
n_latent_factors_mf = 8

In [ ]:
def ecam_neumf():
    # inputs
    item_input = Input(shape=[1],name='item')
    user_input = Input(shape=[1],name='user')
    context_input = Input(shape=(n_context, ), name='context')

    # item embedding MF
    item_embedding_mf = Embedding(n_items + 1, n_latent_factors_mf, name='item_embedding_MF')(item_input)
    item_vec_mf = Flatten(name='flatten_item_MF')(item_embedding_mf)
    item_vec_mf = Dropout(0.2)(item_vec_mf)

    # User embedding MF
    user_embedding_mf = Embedding(n_users + 1, n_latent_factors_mf,name='user_embedding_MF')(user_input)
    user_vec_mf = Flatten(name='flatten_user_MF')(user_embedding_mf)
    user_vec_mf = Dropout(0.2)(user_vec_mf)

    # Dot product MF
    dot = tf.keras.layers.Dot(axes=1)([user_vec_mf, item_vec_mf])

    # Item embedding MLP
    item_embedding_mlp = Embedding(n_items + 1, n_latent_factors_item, name='item_embedding_MLP')(item_input)
    item_vec_mlp = Flatten(name='flatten_item_MLP')(item_embedding_mlp)
    item_vec_mlp = Dropout(0.2)(item_vec_mlp)

    # User embedding MLP
    user_embedding_mlp = Embedding(n_users + 1, n_latent_factors_user,name='user_embedding_MLP')(user_input)
    user_vec_mlp = Flatten(name='flatten_user_MLP')(user_embedding_mlp)
    user_vec_mlp = Dropout(0.2)(user_vec_mlp)

    # Concat user embedding,item embeddings and context vector
    concat = Concatenate(name='user_item_context_MLP')([item_vec_mlp, user_vec_mlp, context_input])

    # dense layers
    dense = Dense(8, name='fully_connected_1')(concat)
    batch_1 = BatchNormalization()(dense)
    dense_2 = Dense(4, name='fully_connected_2')(batch_1)
    batch_2 = BatchNormalization()(dense_2)
    dense_3 = Dense(2, name='fully_connected_3')(batch_2)

    # concat MF and MLP
    concat_mf_mlp = Concatenate(name='MF_MLP')([dense_3, dot])

    # Output
    output = Dense(1, activation='relu',name='Activation')(concat_mf_mlp)

    # make and build the model
    return keras.Model([user_input, item_input, context_input], output)

In [ ]:
ecam_neumf = ecam_neumf()
opt = keras.optimizers.Adam(lr = 0.005)
ecam_neumf.compile(optimizer = opt,loss= 'mean_absolute_error', metrics=['mae', 'mse'])

ecam_neumf.summary()
tf.keras.utils.plot_model(ecam_neumf)

In [ ]:
history = ecam_neumf.fit([train_x.user, train_x.item, train_context], train_y, epochs=15, batch_size=128, verbose=1)

In [ ]:
plot_loss(history)

In [ ]:
pred_y = ecam_neumf.predict([test_x.user, test_x.item, test_context]).flatten()
a = plt.axes(aspect='equal')
plt.scatter(test_y, pred_y)
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [0, 5]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
rmse = mean_squared_error(test_y, pred_y, squared = False)
mse = mean_squared_error(test_y, pred_y, squared = True)
mae = mean_absolute_error(test_y, pred_y)
f'RMSE = {rmse}    MAE = {mae}    MSE = {mse}'

## Latent context extraction
